# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas as hv
import pandas as pd
import requests
import cartopy.crs as ccrs
import holoviews as ho


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buala,-8.1450,159.5921,26.33,85,100,1.52,SB,1698067708
1,1,general pico,-35.6566,-63.7568,18.28,67,100,5.77,AR,1698067708
2,2,port-aux-francais,-49.3500,70.2167,1.77,95,95,10.38,TF,1698067709
3,3,letterkenny,54.9500,-7.7333,13.94,82,100,5.90,IE,1698067709
4,4,lompoc,34.6391,-120.4579,15.85,88,100,5.81,US,1698067611


In [3]:
# Configure the map plot
# Create a Points object using hv.Points with the DataFrame and specifying the latitude and longitude columns
# The size of the point should be the humidity in each city.
map_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)

# Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# .loc subset based on my ideal conditions, remove null values
subset_df = city_data_df.loc[(city_data_df["Max Temp"] > 21.1) 
                             & (city_data_df["Max Temp"] < 35)
                             & (city_data_df["Wind Speed"] < 8)
                             & (city_data_df["Humidity"] < 75)
                             & (city_data_df["Cloudiness"] <= 45), :]
subset_df = subset_df.reset_index()
# check for null and drop any null columns
subset_df.dropna(axis=0)
subset_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,30,saint-pierre,-21.3393,55.4781,28.65,65,0,4.12,RE,1698067715
1,34,34,benghazi,32.1167,20.0667,26.10,52,0,5.11,LY,1698067716
2,38,38,port elizabeth,-33.9180,25.5701,24.62,57,0,4.12,ZA,1698067717
3,46,46,jiwani,25.0500,61.7417,27.59,73,0,5.01,PK,1698067719
4,69,69,niwai,26.3500,75.9167,27.00,29,5,0.31,IN,1698067724


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
col_selection = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
# using [[]] creates an issue of chain indexing, so we use .loc again
hotel_df = subset_df.loc[:, col_selection].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,65,
1,benghazi,LY,32.1167,20.0667,52,
2,port elizabeth,ZA,-33.9180,25.5701,57,
3,jiwani,PK,25.0500,61.7417,73,
4,niwai,IN,26.3500,75.9167,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel",
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
benghazi - nearest hotel: ترجاب
port elizabeth - nearest hotel: Waterford Hotel
jiwani - nearest hotel: No hotel found
niwai - nearest hotel: No hotel found
pomigliano d'arco - nearest hotel: Hotel Fortuna
mormugao - nearest hotel: Hotel Rukhmini
constantia - nearest hotel: Hotel Chérica
tolanaro - nearest hotel: Hôtel Mahavokey
tripoli - nearest hotel: الاصيل
praia da vitoria - nearest hotel: Salles
tamanrasset - nearest hotel: فندق الأمان
kruisfontein - nearest hotel: No hotel found
hammamet - nearest hotel: Résidence L'Arabesque
alice springs - nearest hotel: Aurora Alice Springs
timimoun - nearest hotel: أناف
kankan - nearest hotel: Buffet de la gare
east london - nearest hotel: No hotel found
khategaon - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
sangueya - nearest hotel: No hotel found
vila do maio - nearest hotel: Residencial Jardins De Maio
cidade velha - nearest hotel: Encosta Azul
sal

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,65,Tropic Hotel
1,benghazi,LY,32.1167,20.0667,52,ترجاب
2,port elizabeth,ZA,-33.9180,25.5701,57,Waterford Hotel
3,jiwani,PK,25.0500,61.7417,73,No hotel found
4,niwai,IN,26.3500,75.9167,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    scale = 1.5,
    color = "City",
    hover_cols = ["Country","Hotel Name"]
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)

In [ ]:
#ho.save(map_hotel,"output_data/Fig6(map_hotel).png")
#ho.save(map_humidity,"output_data/Fig5(map_humidity).png")